In [1]:
import pandas as pd
from datetime import datetime
import gzip
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors


## Con el objetivo de aumentar el rendimiendo de la API , al tratarse con un dataset muy grande, se realizaran nuevos dataframe orientados ar esponder consultas de funciones especificas, estos se guardaran en csv y luego seran usados por las funciones


In [2]:
with gzip.open('../datasets/user_items_proc.csv.gz', 'rb') as f:
    user_items_proc = pd.read_csv(f, encoding='utf-8')
    
steam_games = pd.read_csv('../datasets/steam_games.csv')
user_review = pd.read_csv('../datasets/user_reviews.csv')

### Funcion  userdata : Genero un data frame con 4 columnas:
* user_id : con valores unicos.
* intems_count: Cantidad de items.
* price : Total gastado.
* porcentaje_recomendacion_true: Porcentaje de recommend = True frente al total de rocommend

### Luego se guarda en csv y se usara para la funcion

In [3]:


# Hago un merge de  los dataframes 'steam_games' y 'user_items_proc' en 'diner_user' basándonos en 'app_name' y 'item_name'
diner_user = pd.merge(steam_games[['app_name','price']],user_items_proc[['user_id','items_count','item_name']],left_on='app_name',right_on='item_name',how='inner')

# Agrupo los datos en 'users_items' por 'user_id' y agregamos la primera aparición de 'items_count' y la suma de 'price'
users_items = diner_user.groupby('user_id').agg(
    {
       'items_count':'first',
        'price':'sum'
    }
).reset_index()

# Buscamos la suma de recomendaciones por usuario en 'recomendaciones_true'
recomendaciones_true = user_review.groupby('user_id')['recommend'].sum()

# Se cuenta el total de registros por usuario en 'total_registros_por_usuario'
total_registros_por_usuario = user_review.groupby('user_id')['recommend'].count()

# Calculamos el porcentaje de recomendaciones en 'porcentaje_recomendaciones_true'
porcentaje_recomendaciones_true = (recomendaciones_true / total_registros_por_usuario) * 100

# Nuevo dataframe 'users_true_reviews' con 'user_id' y 'porcentaje_recomendaciones_true'
users_true_reviews = pd.DataFrame({
    'user_id': porcentaje_recomendaciones_true.index,
    'porcentaje_recomendaciones_true': porcentaje_recomendaciones_true.values
})

# Uno 'users_items' y 'users_true_reviews' en 'items_reviews_users'
items_reviews_users = pd.merge(users_items,users_true_reviews,on='user_id',how='left')

# Exportamos el dataframe 'items_reviews_users' a un archivo CSV sin índices
items_reviews_users.to_csv('../datasets/items_reviews_users.csv',index=False)

In [4]:
items_reviews_users

,user_id,items_count,price,porcentaje_recomendaciones_true
0,--000--,58,332.83,100.0
1,--ace--,44,122.86,100.0
2,--ionex--,23,69.95,100.0
3,-2SV-vuLB-Kg,68,340.55,100.0
4,-404PageNotFound-,149,917.57,NaN
...,...,...,...,...
68073,zzonci,5,0.00,NaN
68074,zzoptimuszz,61,64.98,100.0
68075,zzydrax,13,84.95,NaN
68076,zzyfo,84,518.64,NaN


### Funcion  UserForGenre : Genero un data frame con 3 columnas:
* Género : Cada uno de los generos.
* Usuario : El usuario que acumula mas horas para ese genero.
* Año_Horas = Una lista de diccionarios con llave como año y cantidad de horas para ese año como valor


### Luego se guarda en csv y se usara para la funcion

In [5]:
user_items_proc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          int64  
 3   user_url          object 
 4   item_id           float64
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(3), int64(2), object(3)
memory usage: 314.5+ MB


In [6]:
# Genero un dataframe con los generos para luego filtrar
user_items_proc['item_id'] = user_items_proc['item_id'].astype(int)
generos = steam_games.drop(columns=['app_name','price','id','developer','Year','Accounting'])

# Unimos 2 datasets usando 'item_name' y 'app_name' como la clave
gener_user = pd.merge(user_items_proc[['item_id','playtime_forever','user_id']],steam_games,left_on='item_id',right_on='id',how='inner')

# Inicializamos un arreglo vacío para almacenar los resultados
resultados_por_genero = []
for genero in generos.columns:
    # Filtramos las filas que corresponden al género actual
    genero_data = gener_user[gener_user[genero] == 1]

    # Comprobamos si existen filas que correspondan al género actual
    if not genero_data.empty:
        # Encontramos el usuario que tiene el valor maximo de 'playtime_forever' para el género actual
        user_max = genero_data.groupby('user_id')['playtime_forever'].sum().div(60).idxmax() # Divido por 60 ya que esta en minitus
        # Agrupamos los datos por año y calculamos la suma de 'playtime_forever'
        result_df = genero_data[genero_data['user_id'] == user_max].groupby('Year')['playtime_forever'].sum().reset_index()
        # Reformateamos los resultados
        formatted_result = [{'Año': record['Year'], 'Horas': record['playtime_forever']} for _, record in result_df.iterrows()]
        # Agregamos los resultados al arreglo
        resultados_por_genero.append({'Género': genero, 'Usuario': user_max, 'Año_Horas': formatted_result})
    else:
        # Si no existen filas que correspondan al género, solo incluimos el genero y el valor nulo para 'usuario' y un arreglo vacío para 'Año_Horas'
        resultados_por_genero.append({'Género': genero, 'Usuario': None, 'Año_Horas': []})

# Convertimos los resultados en un DataFrame
df_resultados = pd.DataFrame(resultados_por_genero)

# Eliminamos las filas con valores faltantes
df_resultados.dropna(inplace=True)

# Guardamos los resultados en un archivo csv
df_resultados.to_csv('../datasets/max_por_gen.csv',index=False)

In [ ]:
df_resultados

,Género,Usuario,Año_Horas
0,Action,Sp3ctre,"[{'Año': 1993.0, 'Horas': 0.0}, {'Año': 1995.0..."
1,Adventure,REBAS_AS_F-T,"[{'Año': 1998.0, 'Horas': 1.0}, {'Año': 1999.0..."
2,Animation &amp; Modeling,ScottyG555,"[{'Año': 2015.0, 'Horas': 168314.0}]"
3,Audio Production,Lickidactyl,"[{'Año': 2014.0, 'Horas': 109916.0}]"
4,Casual,REBAS_AS_F-T,"[{'Año': 1999.0, 'Horas': 0.0}, {'Año': 2002.0..."
5,Design &amp; Illustration,ScottyG555,"[{'Año': 2015.0, 'Horas': 168314.0}]"
6,Early Access,76561197978756659,"[{'Año': 2013.0, 'Horas': 316786.0}, {'Año': 2..."
7,Education,tsunamitad,"[{'Año': 1994.0, 'Horas': 0.0}, {'Año': 1995.0..."
8,Free to Play,shinomegami,"[{'Año': 1996.0, 'Horas': 0.0}, {'Año': 2003.0..."
9,Indie,REBAS_AS_F-T,"[{'Año': 1999.0, 'Horas': 0.0}, {'Año': 2001.0..."


## Funcion con el usuario y el juego en que tiene mas cantidad de horas 

In [4]:
 # hago un merge teniendo todos los usuarios
user_rev_item = pd.merge(user_items_proc[['user_id','item_id','playtime_forever']],user_review[['user_id','recommend']],on = 'user_id',how='left')
# Me quedo solo con los que estan nulos en reviews
user_rev_item = user_rev_item[user_rev_item['recommend'].isnull()]
# Hago un agrupamiento por usuario y saco el id del que tiene mas minuntos.
max_playtime_items = user_rev_item.groupby('user_id')['playtime_forever'].idxmax()
result_df = user_rev_item.loc[max_playtime_items, ['user_id', 'item_id']]
result_df.to_csv('../datasets/user_item_max_horas.csv',index=False) # Exporto para ser usado por la funcion
